# 🧠 LLaMA 3 GPU Inference on RunPod
Setup log for Week 2 – Day 1 of LLM Evaluation Roadmap

In [ ]:
# ✅ 1. System setup (RunPod template: PyTorch 2.1 + CUDA 11.8)
!sudo apt update
!sudo apt install -y cmake build-essential libcurl4-openssl-dev git-lfs wget
!pip install --upgrade pip
!pip install huggingface_hub

In [ ]:
# ✅ 2. Clone and build llama.cpp with CUDA
!cd ~ && git clone https://github.com/ggerganov/llama.cpp.git
!mkdir -p ~/llama.cpp/build
!cd ~/llama.cpp/build && cmake .. -DGGML_CUDA=on -DLLAMA_BUILD_EXAMPLES=on && make -j

In [ ]:
# ✅ 3. Authenticate with Hugging Face and download model
!huggingface-cli login  # You will need to paste your token manually

# Download model (4.6GB) into llama.cpp directory
!mkdir -p ~/llama.cpp/models/llama-3
!huggingface-cli download QuantFactory/Meta-Llama-3-8B-Instruct-GGUF --include "Meta-Llama-3-8B-Instruct.Q4_K_M.gguf" --local-dir ~/llama.cpp/models/llama-3


In [ ]:
# ✅ 4. Run inference!
!~/llama.cpp/build/bin/llama-cli -m ~/llama.cpp/models/llama-3/Meta-Llama-3-8B-Instruct.Q4_K_M.gguf -p "What is the capital of France?" -n 128 --color